# Imports

In [1]:
import numpy as np
import cupy as cp
from numba import cuda, vectorize
from numpy import format_float_scientific as fs
import matplotlib.pyplot as plt
from matplotlib import cm
import time
import math

from lbmFlowAroundCylinder import Timer, TimersManager
from lbmFlowAroundCylinder import inivel, obstacle_fun

## Timers definition

In [2]:
timers = TimersManager()
timers.add("main")
timers.add("equilibrium")
timers.add("collision")
timers.add("streaming")
timers.add("macroscopic")
timers.add("rightwall")
timers.add("leftwall")
timers.add("fineq")
timers.add("bounceback")
timers.add("move_gpu->cpu")
timers.add("move_cpu->gpu")

## Flow definitions

In [3]:
maxIter = 2000    # Total number of time iterations.
Re = 150.0          # Reynolds number.
nx, ny = 420, 180   # Numer of lattice nodes.
ly = ny-1           # Height of the domain in lattice units.
cx, cy, r = nx//4, ny//2, ny//9 # Coordinates of the cylinder.
uLB     = 0.04                  # Velocity in lattice units.
nulb    = uLB*r/Re;             # Viscoscity in lattice units.
omega = 1 / (3*nulb+0.5);    # Relaxation parameter.
save_figures = False
profile = True

## Lattice constants

In [4]:
v = cuda.to_device(np.array([ [ 1,  1], [ 1,  0], [ 1, -1], [ 0,  1], [ 0,  0],
               [ 0, -1], [-1,  1], [-1,  0], [-1, -1] ], dtype=np.int32)) # 9 vecteurs : 9 directions de déplacement
v_np = np.empty(shape=v.shape, dtype=v.dtype)
v.copy_to_host(v_np)

t = cuda.to_device(np.array([ 1/36, 1/9, 1/36, 1/9, 4/9, 1/9, 1/36, 1/9, 1/36], 
                            dtype=np.float32))
t_np = np.empty(shape=t.shape, dtype=t.dtype)
t.copy_to_host(t_np)

col1 = np.array([0, 1, 2])
col2 = np.array([3, 4, 5])
col3 = np.array([6, 7, 8])

# Code main functions

### Macroscopic

In [5]:
@cuda.jit
def sum_kernel_axis0(A, out):
    """ Performs the sum on axis 0
    """
    y, z = cuda.grid(2)

    if y < A.shape[1] and z < A.shape[2]:
        tmp = 0
        for i in range(A.shape[0]):
            tmp += A[i, y, z]
        out[y, z] = tmp

def macroscopic(fin): 
    """Compute macroscopic variables (density, velocity)

    fluid density is 0th moment of distribution functions 
    fluid velocity components are 1st order moments of dist. functions
    """
    rho = np.sum(fin, axis=0)
    u = np.zeros((2, nx, ny))
    for i in range(9):
        u[0,:,:] += v[i,0] * fin[i,:,:]
        u[1,:,:] += v[i,1] * fin[i,:,:]
    u /= rho
    return rho, u
        
        
mac_threadsperblock = (16, 16)
mac_blockspergrid_x = math.ceil(nx / mac_threadsperblock[0])
mac_blockspergrid_y = math.ceil(ny / mac_threadsperblock[1])
mac_blockspergrid = (mac_blockspergrid_x, mac_blockspergrid_y)


@cuda.jit
def macroscopic_cuda(fin, v, rho_out, u_out):
    x, y = cuda.grid(2)
    if x < rho_out.shape[0] and y < rho_out.shape[1]:
        rho_tmp = 0
        ux_tmp = 0
        uy_tmp = 0
        for ipop in range(9):
            rho_tmp += fin[ipop, x, y]
            ux_tmp += v[ipop, 0] * fin[ipop, x, y]
            uy_tmp += v[ipop, 0] * fin[ipop, x, y]
        rho_out[x, y] = rho_tmp
        u_out[0, x, y] = ux_tmp / rho_tmp
        u_out[1, x, y] = uy_tmp / rho_tmp

### Equilibrium

In [6]:
def equilibrium(rho, u):
    """Equilibrium distribution function.
    """
    usqr = 3/2 * (u[0]**2 + u[1]**2)
    feq = np.zeros((9,nx,ny))
    for i in range(9):
        cu = 3 * (v[i,0]*u[0,:,:] + v[i,1]*u[1,:,:])
        feq[i,:,:] = rho*t[i] * (1 + cu + 0.5*cu**2 - usqr) 
        # feq[i,:,:] : dimension 1 la direction de déplacement de la particule
        #               dimension 2 et 3 : x et y la position
    return feq

equ_threadsperblock = (16, 16)
equ_blockspergrid_x = math.ceil(nx / equ_threadsperblock[0])
equ_blockspergrid_y = math.ceil(ny / equ_threadsperblock[1])
equ_blockspergrid   = (equ_blockspergrid_x, equ_blockspergrid_y)

@cuda.jit
def equilibrium_cuda(rho, u, v, t, feq_out):
    x, y = cuda.grid(2)
    if x < rho.shape[0] and y < rho.shape[1]:
        ux = u[0, x, y]
        uy = u[1, x, y]
        usqr = 1.5 * (ux * ux + uy * uy) 
        for ipop in range(9):
            cu = 3 * (v[ipop, 0] * ux + v[ipop, 1] * uy)
            feq_out[ipop, x, y] = rho[x, y] * t[ipop] * (1 + cu + 0.5 * cu * cu - usqr)

## BounceBack

In [7]:
bou_threadsperblock = (16, 16)
bou_blockspergrid_x = math.ceil(nx / bou_threadsperblock[0])
bou_blockspergrid_y = math.ceil(ny / bou_threadsperblock[1])
bou_blockspergrid   = (bou_blockspergrid_x, bou_blockspergrid_y)


@cuda.jit
def bounceback_cuda(fin, obstacle, f_out):
    x, y = cuda.grid(2)
    if x < obstacle.shape[0] and y < obstacle.shape[1]:
        if obstacle[x, y] == 1:
            for i in range(9):
                f_out[i, x, y] = fin[8 - i, x, y]

### Main loop

In [8]:
def main():
    # create obstacle mask array from element-wise function
    obstacle = np.fromfunction(obstacle_fun, (nx,ny))
    obstacle_device = cuda.to_device(obstacle)
    
    # initial velocity field vx,vy from element-wise function
    # vel is also used for inflow border condition
    vel = np.fromfunction(inivel, (2,nx,ny)) 
    
    # Initialization of the populations at equilibrium 
    # with the given velocity.
    fin = equilibrium(1, vel)
    fin_device = cuda.to_device(fin)
    
    rho = np.zeros(shape=(fin.shape[1], fin.shape[2]))
    rho_device = cuda.to_device(rho)
    
    u = np.zeros((2, nx, ny), dtype=np.float32)
    u_device = cuda.to_device(u)
    
    feq = np.zeros_like(fin)
    feq_device = cuda.to_device(feq)
    
    fout = np.zeros_like(fin)
    fout_device = cuda.to_device(fout)
    

    ###### Main time loop ########
    for time in range(maxIter):
        # Right wall: outflow condition.
        # we only need here to specify distrib. function for velocities
        # that enter the domain (other that go out, are set by the streaming step)
        

        timers.get("rightwall").start()
        fin[col3,nx-1,:] = fin[col3,nx-2,:] 
        timers.get("rightwall").end()

        
        # Compute macroscopic variables, density and velocity.
        timers.get("macroscopic").start()
        #macroscopic_cuda[mac_blockspergrid, mac_threadsperblock](fin, v, rho, u) # Timer in func
        rho, u = macroscopic(fin)
        timers.get("macroscopic").end()

        # Left wall: inflow condition.
        timers.get("leftwall").start()
        u[:,0,:] = vel[:,0,:]
        rho[0,:] = 1/(1-u[0,0,:]) * ( np.sum(fin[col2,0,:], axis=0) +
                                      2*np.sum(fin[col3,0,:], axis=0) )
        timers.get("leftwall").end()
        
        timers.get("move_cpu->gpu").start()
        rho_device = cuda.to_device(rho)
        u_device = cuda.to_device(u)
        feq_device = cuda.to_device(feq)
        timers.get("move_cpu->gpu").end()
        
        # Compute equilibrium.
        timers.get("equilibrium").start()
        equilibrium_cuda[equ_blockspergrid, equ_threadsperblock](rho_device, u_device, v, t, feq_device) # Timer in func
        #feq = equilibrium(rho, u)
        timers.get("equilibrium").end()
    
        timers.get("move_gpu->cpu").start()
        rho_device.copy_to_host(rho)
        u_device.copy_to_host(u)
        feq_device.copy_to_host(feq)
        timers.get("move_gpu->cpu").end()
    
        timers.get("fineq").start()
        fin[[0,1,2],0,:] = feq[[0,1,2],0,:] + fin[[8,7,6],0,:] - feq[[8,7,6],0,:]
        timers.get("fineq").end()

        # Collision step.
        timers.get("collision").start()
        fout = fin - omega * (fin - feq) # Noyau de calcul 1
        timers.get("collision").end()

        
        timers.get("move_cpu->gpu").start()
        fin_device = cuda.to_device(fin)
        fout_device = cuda.to_device(fout)
        timers.get("move_cpu->gpu").end()
        
        # Bounce-back condition for obstacle.
        # in python language, we "slice" fout by obstacle
        timers.get("bounceback").start()
        bounceback_cuda[bou_blockspergrid, bou_threadsperblock](fin_device, obstacle_device, fout_device)
        #for i in range(9):
        #    fout[i, obstacle] = fin[8-i, obstacle]
        timers.get("bounceback").end()

        timers.get("move_gpu->cpu").start()
        fin_device.copy_to_host(fin)
        fout_device.copy_to_host(fout)
        timers.get("move_gpu->cpu").end()
        
        # Streaming step.
        timers.get("streaming").start()
        for i in range(9):
            fin[i,:,:] = np.roll(np.roll(fout[i,:,:], v_np[i,0], axis=0),
                                 v_np[i,1], axis=1 ) # Noyau de calcul 2
        timers.get("streaming").end()
        
        if ((time%100==0) and save_figures):
            plt.clf()
            #u_host = np.zeros_like(u)
            #u.copy_to_host(u_host)
            plt.imshow(np.sqrt(u[0]**2+u[1]**2).transpose(), cmap=cm.Reds)
            plt.show()
            #plt.savefig("figures/vel.{0:04d}.png".format(time//100))


In [9]:
timers.get("main").start()
main()
timers.get("main").end()

In [10]:
total = np.sum(timers.get("main").getMeasures())
print(f"Total time : {total:4.2f}s")
timers.printInfo()

Total time : 24.35s
--> Timer 'main         ' : N =    1 | Mean 2.435e+01 +- 0.e+00     | 100.0% of total time.
--> Timer 'equilibrium  ' : N = 2000 | Mean 3.110e-04 +- 6.23e-03   |  2.55% of total time.
--> Timer 'collision    ' : N = 2000 | Mean 1.258e-03 +- 4.615e-05  | 10.33% of total time.
--> Timer 'streaming    ' : N = 2000 | Mean 1.282e-03 +- 2.058e-05  | 10.53% of total time.
--> Timer 'macroscopic  ' : N = 2000 | Mean 2.699e-03 +- 5.995e-05  | 22.17% of total time.
--> Timer 'rightwall    ' : N = 2000 | Mean 1.686e-05 +- 1.311e-06  |  0.14% of total time.
--> Timer 'leftwall     ' : N = 2000 | Mean 4.272e-05 +- 1.754e-06  |  0.35% of total time.
--> Timer 'fineq        ' : N = 2000 | Mean 3.648e-05 +- 4.355e-06  |   0.3% of total time.
--> Timer 'bounceback   ' : N = 2000 | Mean 2.338e-04 +- 1.259e-03  |  1.92% of total time.
--> Timer 'move_gpu->cpu' : N = 4000 | Mean 1.115e-03 +- 7.46e-05   | 18.31% of total time.
--> Timer 'move_cpu->gpu' : N = 4000 | Mean 2.010e-03 +- 6.1

## Tests